In [1]:
!pip install nlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00


In [2]:
!pip install transformers tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 837.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
pip install transformers[torch] --upgrade -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.9 MB/s eta 0:00:00


In [4]:
pip install accelerate -U -q


In [5]:
pip install datasets -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [45]:
import nlp
import datasets
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline


In [7]:
ds = datasets.load_dataset('conll2003')


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [9]:
train = ds['train']
valid = ds['validation']
test = ds['test']


In [10]:
train[0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [11]:
train.features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [12]:
train.description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [13]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [63]:
unique_ner_tags = set()
for example in train:
    unique_ner_tags.update(example['ner_tags'])

num_classes = len(unique_ner_tags)


In [64]:
num_classes

9

In [14]:
example = train[0]
tokenized_input = tokenizer(example['tokens'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
word_ids = tokenized_input.word_ids()
word_ids


[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [15]:
tokens

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [16]:
len(example['ner_tags']), len(tokenized_input['input_ids'])

(9, 11)

In [17]:
def tokenize_and_align_labels(example, label_all_tokens = True):
  tokenized_input = tokenizer(example['tokens'],
                              truncation = True,
                              is_split_into_words = True)
  labels = []

  for i, label in enumerate(example['ner_tags']):
    word_ids = tokenized_input.word_ids(batch_index=i)
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(label[word_idx] if label_all_tokens else -100)
      previous_word_idx = word_idx

    labels.append(label_ids)
  tokenized_input['labels'] = labels
  return tokenized_input


In [18]:
q = tokenize_and_align_labels(train[0:1])
print(q)

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]]}


In [19]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]), q['labels'][0]):
  print(f"{token:<40}{label}")


[CLS]                                   -100
eu                                      3
rejects                                 0
german                                  7
call                                    0
to                                      0
boycott                                 0
british                                 7
lamb                                    0
.                                       0
[SEP]                                   -100


In [20]:
tokenized_ds = ds.map(tokenize_and_align_labels, batched = True)


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [21]:
model = AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels = 9 )


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = 'epoch',
    learning_rate = 0.0001,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 1,
    weight_decay=0.01
)

In [23]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [24]:
metric = datasets.load_metric('seqeval')

<ipython-input-24-8729a7e2f3c1>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('seqeval')


In [25]:
label_list = train.features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [26]:
labels = [label_list[i] for i in example['ner_tags']]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [27]:
metric.compute(predictions=[labels], references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [38]:
def compute_metrics(eval_preds):
  pred_logits, labels = eval_preds
  pred_logits = np.argmax(pred_logits, axis=2)
  prediction = [
      [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)
  ]

  true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)
  ]

  results = metric.compute(predictions = prediction, references = true_labels)

  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"]
  }

In [39]:
trainer = Trainer(
    model,args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()
model.save_pretrained('ner_model')
tokenizer.save_pretrained('tokenizer')

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.016500,0.066493,0.932432,0.941716,0.937051,0.985051


Checkpoint destination directory test-ner/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [41]:
id2label = {
    str(i) : label for i, label in enumerate(label_list)
}
label2id = {
    label: str(i) for i, label in enumerate(label_list)
}

In [42]:
import json
config = json.load(open('ner_model/config.json'))

In [43]:
config['id2label'] = id2label
config['label2id'] = label2id

json.dump(config, open('ner_model/config.json', 'w'))

fine_tuned_model = AutoModelForTokenClassification.from_pretrained('ner_model')

In [46]:
fine_tuned_model = AutoModelForTokenClassification.from_pretrained("ner_model")

In [60]:
nlp = pipeline('ner', model=fine_tuned_model, tokenizer=tokenizer)

example1 = "Jennifer works at Microsoft's headquarters in Seattle."

ner_result = nlp(example1)
print(ner_result)

[{'entity': 'B-PER', 'score': 0.9992824, 'index': 1, 'word': 'jennifer', 'start': 0, 'end': 8}, {'entity': 'B-ORG', 'score': 0.9953921, 'index': 4, 'word': 'microsoft', 'start': 18, 'end': 27}, {'entity': 'B-LOC', 'score': 0.9989497, 'index': 9, 'word': 'seattle', 'start': 46, 'end': 53}]
